## Clase 2

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df_ratings = pd.read_csv('./data/ratings_train.csv')
df_test = pd.read_csv('./data/ratings_test.csv')

In [3]:
df_ratings.head()

,ID,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,1,51198,608,5.0,17,9,1997,23,40,20
1,2,51198,141,4.0,17,9,1997,23,41,58
2,3,51198,1073,4.0,17,9,1997,23,41,6
3,4,51198,356,4.0,17,9,1997,23,43,44
4,5,51198,1198,5.0,17,9,1997,23,44,35


In [4]:
df_movie_mean = df_ratings.groupby('movieID')['rating'].mean().reset_index()
df_movie_mean.rename(columns={'rating': 'mean_rating'}, inplace=True)

In [5]:
df_ratings = df_ratings.merge(df_movie_mean, how='left', left_on='movieID', right_on='movieID')
df_test = df_test.merge(df_movie_mean, how='left', left_on='movieID', right_on='movieID')

In [6]:
df_test.fillna(df_ratings['rating'].mean(), inplace=True)

In [10]:
df_test.head()

,ID,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second,mean_rating
0,9,29202,1608,3.438057,18,9,1997,0,53,58,3.055254
1,18,32283,1485,3.438057,18,9,1997,15,45,18,3.070827
2,34,32283,30,3.438057,18,9,1997,16,36,38,3.777778
3,39,32283,1545,3.438057,18,9,1997,16,47,39,4.000000
4,50,51198,1258,3.438057,20,9,1997,0,11,21,3.964068


In [7]:
df_ratings.to_csv('./data/ratings_train_with_mean.csv', index=False)
df_test.to_csv('./data/ratings_test_with_mean.csv', index=False)

In [24]:
# Separo mi train & test
X_train, X_test, y_train, y_test =\
    train_test_split(df_ratings.drop(['rating', 'ID'], axis=1), df_ratings.rating, test_size=0.3, random_state=0)

# Entrenamiento regresion lineal
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
scores = model.score(X_test, y_test)
print("Linear regression: ", mean_squared_error(y_test, y_predicted))
print("Linear regression R2: ", r2_score(y_test, y_predicted))

Linear regression:  0.7585420067199617
Linear regression R2:  0.2474867592723371


In [25]:
model.fit(df_ratings.drop(['rating', 'ID'], axis=1), df_ratings.rating)
prediction = model.predict(df_test.drop(['rating', 'ID', 'rating'], axis=1))
submission = pd.DataFrame(
        {'ID': df_test.ID, 'rating': np.around(prediction, 2)})
submission.to_csv('./submission_mean_rating.csv', index=False)